# Restoration and degradation widget
##### The purpose of this notebook is to format the Typology_and_Restoration_Potential table to produce tables that can be used by the API. The data models we need should have the following format:

**Restoration potential:**  
{  
location_id [str]  
indicator [str] - restorable_area, mangrove_area, restoration_potential_score  
value [numeric]  
unit [str] - ha, %  
}  

where:

restorable_area = Tot_Restor  
mangrove_area  = Total_2016  
restoration_potential_score  =  Mean_Score  

**Degradation:**  
{  
location_id [str]  
indicator [str] - degraded_area, lost_area, main_loss_driver  
value [numeric]  
unit [str] - ha, %  
}  

where:

degraded_area = Rest_Area_Dgrd  
lost_area = Area_loss_ha  
main_loss_driver = Loss_Driver


### Steps:
1. Import packages and data
2. Format restoration potential table:
- Get countries for the table
- Give each country their corresponding iso code
- Add staging ids
- Create indicators
- Table cleaning
- Save table
3. Format degradation table:
- Get countries for the table
- Give each country their corresponding iso code
- Add staging ids
- Create indicators
- Table cleaning
- Save table

#### 1. Import packages and data

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import fiona


In [2]:
# Import geodatabase and get Global_Mangrove_Restoration_Jurisdiction_Envelope
#path_in = '../../../../data/Typology_and_Restoration_Potential/Data'
#gdb_file =f'{path_in}/MOW_Global_Mangrove_Restoration_20190411.gdb'
#layers = fiona.listlayers(gdb_file)
#layer0 = gpd.read_file(gdb_file, driver='FileGDB', layer=0)

path_in = 'https://storage.googleapis.com/mangrove_atlas/widget_data'
data_df = pd.read_csv(f'{path_in}/Vizz_Country_Stats_Dec2022.csv', index_col=0)
data_df.head()


,Name,Total Mangrove Area 2020 (ha),Total Area of Loss (ha),Total Area Restorable (ha),OVERALL_SRE,Total_Fish,Fish,Crab,Bivalve,Shrimp,Total AGC (mtCO2e) from restored areas,Total SOC (mtCO2e) from restored areas,Proportion_Restorable,Extreme_Weather,Non_Productive,Commodities,Driver
1,American Samoa,32.004317,0.594951,0.442066,0.830489,9.129377e+03,9081.125331,4.825178e+01,0.0,0.0,82.919195,144.531066,1.38,0.000619,0.000594,0.003208,Commodities
2,Angola,28319.919690,1546.820441,856.958013,0.692000,4.553915e+04,45539.145920,0.000000e+00,0.0,0.0,110509.432500,234265.161100,3.03,0.254498,8.113579,0.000000,Non-Productive Conversion
3,Anguilla,3.836882,1.571377,1.571377,0.620000,2.378690e+04,10165.451040,1.362145e+04,0.0,0.0,11.015105,461.861506,40.95,0.000000,0.015714,0.000000,Non-Productive Conversion
4,Antigua and Barbuda,871.859248,107.809050,107.809050,0.632000,2.045093e+06,873980.752300,1.171113e+06,0.0,0.0,2714.655158,28326.766570,12.37,0.000000,1.078090,0.000000,Non-Productive Conversion
5,Aruba,47.371025,8.345105,5.643541,0.597168,8.612742e+04,36806.980660,4.932044e+04,0.0,0.0,185.684321,1142.345668,11.91,0.016808,0.034860,0.004767,Non-Productive Conversion


### Add (staging) locations

In [3]:
locations_file = 'https://storage.googleapis.com/mangrove_atlas/boundaries/processed/location_final/locations_v3_not_merged_with_old.gpkg'
locations = gpd.read_file(locations_file)
locations = locations[locations['type'] == 'country']
locations.head()

,name,iso,type,area_m2,wdpaid,globalid,perimeter_m,location_idn,coast_length_m,geometry
82,Qatar,QAT,country,3.880224,NaN,{AF97ABE2-6405-4438-A7ED-1494A43DA379},8.392644,06d2e6f9-bc89-59bf-a0e2-ab804e5db9fd,1345769.96,"MULTIPOLYGON (((50.73769 24.93464, 50.73779 24..."
89,Mayotte,MYT,country,5.611808,NaN,{57E86B5B-7EF0-4754-A8D4-A9DC3212D421},10.086238,0750953f-4af9-549b-aeea-329663249a56,291036.71,"POLYGON ((46.63483 -12.96039, 46.63197 -12.969..."
118,Vietnam,VNM,country,90.156489,NaN,{B2A84FBB-34CD-4A51-9463-B9DB2DB62A10},81.714911,09a1ab14-11ad-56ec-8acb-a149e5697abd,9005760.08,"MULTIPOLYGON (((104.31952 10.36051, 104.31975 ..."
132,Grenada,GRD,country,2.154728,NaN,{F8753179-5FFA-4D9E-8AD9-083F31C48528},6.743601,0b0ecb56-bb8e-5ef1-b8ee-3cdad67fed0e,260664.47,"MULTIPOLYGON (((-61.91525 11.37330, -61.91813 ..."
149,India,IND,country,473.029671,NaN,{A4A6CE4D-8D03-4246-9A2F-BD9811232115},211.564078,0c07ca53-7b17-5650-a2c6-0cc27249a4bd,16917891.22,"MULTIPOLYGON (((79.52922 9.38411, 79.52921 9.3..."


In [4]:
api_locs = pd.read_csv('https://storage.googleapis.com/mangrove_atlas/widget_data/locations_staging.csv')
api_locs.rename(columns={'location_id': 'location_idn'}, inplace=True)
api_locs.head()

,id,location_idn
0,1563,000bd204-c0fd-510b-a1ad-132a7ef7470d
1,1564,00250a0f-f66d-54a0-b7a3-d80035881cbf
2,1565,0041637b-f6a2-5b89-87ce-850f5c5431b3
3,1566,005b49ef-6b7f-575a-85b3-ff19261a0755
4,1567,00921349-70fb-5a7e-8207-b3157aecc349


In [5]:
# Import gadm file to retrieve ISO codes
#gadm = gpd.read_file('../../../../data/gadm36_level0_original/gadm36_level0_original.shp')

#Get ISO codes from locations file
isocodes = locations[['name', 'iso']].copy()
isocodes.head()

,name,iso
82,Qatar,QAT
89,Mayotte,MYT
118,Vietnam,VNM
132,Grenada,GRD
149,India,IND


#### 2. Create Restoration Potential table
Get countries for the table


In [6]:
rp = data_df[['Name']]
rp.head()

,Name
1,American Samoa
2,Angola
3,Anguilla
4,Antigua and Barbuda
5,Aruba


Give each country their corresponding iso code


In [7]:
# Add iso codes to countries in rp dataframe
rp = pd.merge(rp, isocodes, how='left', left_on='Name', right_on='name')
rp.head()

,Name,name,iso
0,American Samoa,American Samoa,ASM
1,Angola,Angola,AGO
2,Anguilla,Anguilla,AIA
3,Antigua and Barbuda,Antigua and Barbuda,ATG
4,Aruba,Aruba,ABW


In [8]:
# Check which countries did not match with gadm and have no iso
[rp['iso'].isnull()]

[0      False
 1      False
 2      False
 3      False
 4      False
        ...  
 117    False
 118    False
 119    False
 120    False
 121    False
 Name: iso, Length: 122, dtype: bool]

In [9]:
locations[locations['name'].str.contains('Ivo')][['name', 'iso']]

,name,iso
1969,Côte d'Ivoire,CIV


In [10]:
# Add isocodes to those countries
rp.loc[rp.Name == 'Mexico', 'iso'] = 'MEX'
rp.loc[rp.Name == 'Cote d\'Ivoire', 'iso'] = 'CIV'
rp.loc[rp.Name == 'Congo', 'iso'] = 'COG'
rp.loc[rp.Name == 'Sao Tome and Principe', 'iso'] = 'STP'

In [11]:
rp = rp.drop(columns='name')
rp.head()

,Name,iso
0,American Samoa,ASM
1,Angola,AGO
2,Anguilla,AIA
3,Antigua and Barbuda,ATG
4,Aruba,ABW


Add staging ids

In [12]:
# Add codes from locations
rp_locs = pd.merge(rp, locations[['iso', 'location_idn']], on='iso', how='left')
rp_locs = pd.merge(rp_locs, api_locs, on='location_idn', how='left')
#mang_df_final = mang_df_final[mang_df_final.id.notna()]
rp_locs.drop(columns=['location_idn'], inplace=True)
rp_locs

,Name,iso,id
0,American Samoa,ASM,2346
1,Angola,AGO,2029
2,Anguilla,AIA,1915
3,Antigua and Barbuda,ATG,3095
4,Aruba,ABW,2707
...,...,...,...
117,Vietnam,VNM,1681
118,British Virgin Islands,VGB,3037
119,"Virgin Islands, U.S.",VIR,2339
120,Wallis and Futuna,WLF,4170


In [13]:
# Countries with no ID
rp_locs[rp_locs['id'].isnull()]

,Name,iso,id


Create indicators

In [14]:
data_df.columns

Index(['Name', 'Total Mangrove Area 2020 (ha)', 'Total Area of Loss (ha)',
       'Total Area Restorable (ha)', 'OVERALL_SRE', 'Total_Fish', 'Fish',
       'Crab', 'Bivalve', 'Shrimp', 'Total AGC (mtCO2e) from restored areas',
       'Total SOC (mtCO2e) from restored areas', 'Proportion_Restorable',
       'Extreme_Weather', 'Non_Productive', 'Commodities', 'Driver'],
      dtype='object')

In [15]:
# Add restorable area indicator
ind1 = rp_locs.copy()
ind1['indicator'] = 'restorable_area'
ind1 = pd.merge(ind1,data_df[['Name','Total Area Restorable (ha)']],on='Name', how='left').rename(columns={'Total Area Restorable (ha)':'value'})
ind1['unit']='ha'
ind1.head()

,Name,iso,id,indicator,value,unit
0,American Samoa,ASM,2346,restorable_area,0.442066,ha
1,Angola,AGO,2029,restorable_area,856.958013,ha
2,Anguilla,AIA,1915,restorable_area,1.571377,ha
3,Antigua and Barbuda,ATG,3095,restorable_area,107.809050,ha
4,Aruba,ABW,2707,restorable_area,5.643541,ha


In [16]:
# Add mangrove area indicator
ind2 = rp_locs.copy()
ind2['indicator'] = 'mangrove_area'
ind2 = pd.merge(ind2,data_df[['Name','Total Mangrove Area 2020 (ha)']],on='Name', how='left').rename(columns={'Total Mangrove Area 2020 (ha)':'value'})
ind2['unit']='ha'
ind2.head()

,Name,iso,id,indicator,value,unit
0,American Samoa,ASM,2346,mangrove_area,32.004317,ha
1,Angola,AGO,2029,mangrove_area,28319.919690,ha
2,Anguilla,AIA,1915,mangrove_area,3.836882,ha
3,Antigua and Barbuda,ATG,3095,mangrove_area,871.859248,ha
4,Aruba,ABW,2707,mangrove_area,47.371025,ha


In [17]:
# Add restoration potential score indicator
ind3 = rp_locs.copy()
ind3['indicator'] = 'restoration_potential_score'
ind3 = pd.merge(ind3,data_df[['Name','OVERALL_SRE']],on='Name', how='left').rename(columns={'OVERALL_SRE':'value'})
ind3['value'] = round(ind3['value']*100, 0)
ind3['unit']=''
ind3.head()

,Name,iso,id,indicator,value,unit
0,American Samoa,ASM,2346,restoration_potential_score,83.0,
1,Angola,AGO,2029,restoration_potential_score,69.0,
2,Anguilla,AIA,1915,restoration_potential_score,62.0,
3,Antigua and Barbuda,ATG,3095,restoration_potential_score,63.0,
4,Aruba,ABW,2707,restoration_potential_score,60.0,


In [25]:
# Merge all indicators in same table
restoration = pd.concat([ind1, ind2, ind3], ignore_index=True)
restoration


,Name,iso,id,indicator,value,unit
0,American Samoa,ASM,2346,restorable_area,0.442066,ha
1,Angola,AGO,2029,restorable_area,856.958013,ha
2,Anguilla,AIA,1915,restorable_area,1.571377,ha
3,Antigua and Barbuda,ATG,3095,restorable_area,107.809050,ha
4,Aruba,ABW,2707,restorable_area,5.643541,ha
...,...,...,...,...,...,...
361,Vietnam,VNM,1681,restoration_potential_score,76.000000,
362,British Virgin Islands,VGB,3037,restoration_potential_score,60.000000,
363,"Virgin Islands, U.S.",VIR,2339,restoration_potential_score,60.000000,
364,Wallis and Futuna,WLF,4170,restoration_potential_score,NaN,


In [30]:
country_name = "Indonesia"
restoration[restoration['Name'].str.contains(country_name)]

,Name,iso,id,indicator,value,unit
54,Indonesia,IDN,3384,restorable_area,2.045362e+05,ha
176,Indonesia,IDN,3384,mangrove_area,2.953651e+06,ha
298,Indonesia,IDN,3384,restoration_potential_score,7.900000e+01,


Table cleaning

In [22]:
# Countries with no staging id
restoration[restoration.id.isna()]

,Name,iso,id,indicator,value,unit


In [23]:
restoration_final = restoration [['id', 'indicator', 'value', 'unit']].copy()
restoration_final.unit = 'ha'
#restoration_final['year'] = 2020
restoration_final.rename(columns={'id':'location_id'}, inplace=True)
restoration_final.head(2)


,location_id,indicator,value,unit
0,2346,restorable_area,0.442066,ha
1,2029,restorable_area,856.958013,ha


Save table

In [24]:
restoration_final[~restoration_final.location_id.isna()].to_csv('../../../../data/RestorationPotential_202212.csv', index = False)

#### Format degradation table


Create indicators

In [23]:
data_df.columns

Index(['Name', 'Total Mangrove Area 2020 (ha)', 'Total Area of Loss (ha)',
       'Total Area Restorable (ha)', 'OVERALL_SRE', 'Total_Fish', 'Fish',
       'Crab', 'Bivalve', 'Shrimp', 'Total AGC (mtCO2e) from restored areas',
       'Total SOC (mtCO2e) from restored areas', 'Proportion_Restorable',
       'Extreme_Weather', 'Non_Productive', 'Commodities', 'Driver'],
      dtype='object')

In [24]:
# Create degraded area indicator
ind1 = rp_locs.copy()
ind1['indicator'] = 'degraded_area'
#Degraded area = Loss - Restorable
ind1 = pd.merge(ind1,data_df[['Name','Total Area of Loss (ha)', 'Total Area Restorable (ha)']],on='Name', how='left')
ind1['value'] = ind1['Total Area of Loss (ha)'] - ind1['Total Area Restorable (ha)']
ind1['unit']='ha'
ind1.drop(columns=['Total Area of Loss (ha)', 'Total Area Restorable (ha)'], inplace = True)
ind1.head()

,Name,iso,id,indicator,value,unit
0,American Samoa,ASM,2346,degraded_area,0.152885,ha
1,Angola,AGO,2029,degraded_area,689.862428,ha
2,Anguilla,AIA,1915,degraded_area,0.000000,ha
3,Antigua and Barbuda,ATG,3095,degraded_area,0.000000,ha
4,Aruba,ABW,2707,degraded_area,2.701563,ha


In [25]:
# Create lost area indicator
ind2 = rp_locs.copy()
ind2['indicator'] = 'lost_area'
ind2 = pd.merge(ind2,data_df[['Name','Total Area of Loss (ha)']],on='Name', how='left').rename(columns={'Total Area of Loss (ha)':'value'})
ind2['unit']='ha'
ind2.head()

,Name,iso,id,indicator,value,unit
0,American Samoa,ASM,2346,lost_area,0.594951,ha
1,Angola,AGO,2029,lost_area,1546.820441,ha
2,Anguilla,AIA,1915,lost_area,1.571377,ha
3,Antigua and Barbuda,ATG,3095,lost_area,107.809050,ha
4,Aruba,ABW,2707,lost_area,8.345105,ha


In [26]:
# Add mangrove area indicator
ind3 = rp_locs.copy()
ind3['indicator'] = 'mangrove_area'
ind3 = pd.merge(ind3,data_df[['Name','Total Mangrove Area 2020 (ha)']],on='Name', how='left').rename(columns={'Total Mangrove Area 2020 (ha)':'value'})
ind3['unit']='ha'
ind3.head()

,Name,iso,id,indicator,value,unit
0,American Samoa,ASM,2346,mangrove_area,32.004317,ha
1,Angola,AGO,2029,mangrove_area,28319.919690,ha
2,Anguilla,AIA,1915,mangrove_area,3.836882,ha
3,Antigua and Barbuda,ATG,3095,mangrove_area,871.859248,ha
4,Aruba,ABW,2707,mangrove_area,47.371025,ha


In [27]:
# Add mangrove area indicator
ind4 = rp_locs.copy()
ind4['indicator'] = 'restorable_area'
ind4 = pd.merge(ind4,data_df[['Name','Total Area Restorable (ha)']],on='Name', how='left').rename(columns={'Total Area Restorable (ha)':'value'})
ind4['unit']='ha'
ind4.head()

,Name,iso,id,indicator,value,unit
0,American Samoa,ASM,2346,restorable_area,0.442066,ha
1,Angola,AGO,2029,restorable_area,856.958013,ha
2,Anguilla,AIA,1915,restorable_area,1.571377,ha
3,Antigua and Barbuda,ATG,3095,restorable_area,107.809050,ha
4,Aruba,ABW,2707,restorable_area,5.643541,ha


In [48]:
# Create main loss driver indicator
ind5 = rp_locs.copy()
ind5['indicator'] = 'main_loss_driver'
ind5 = pd.merge(ind5,data_df[['Name','Driver']],on='Name', how='left').rename(columns={'Driver':'main_loss_driver'})
ind5= ind5.replace({'main_loss_driver':{'No Data':'undefined'}})
ind5.loc[ind5.main_loss_driver.isna(), 'main_loss_driver'] = 'undefined'
ind5 = ind5[['iso', 'main_loss_driver']]
ind5.head(8)

,iso,main_loss_driver
0,ASM,Commodities
1,AGO,Non-Productive Conversion
2,AIA,Non-Productive Conversion
3,ATG,Non-Productive Conversion
4,ABW,Non-Productive Conversion
5,AUS,Extreme Weather
6,BHS,Extreme Weather
7,BHR,Non-Productive Conversion


In [49]:
ind5.main_loss_driver.value_counts()

Non-Productive Conversion    66
Commodities                  26
Extreme Weather              18
undefined                    12
Name: main_loss_driver, dtype: int64

In [50]:
# Merge all indicators in same table
degradation = pd.concat([ind1, ind2, ind3,ind4], ignore_index=True)
degradation_driver = pd.merge(degradation, ind5, on='iso', how = 'left')
degradation_driver['year'] = 2020
degradation_driver

,Name,iso,id,indicator,value,unit,main_loss_driver,year
0,American Samoa,ASM,2346,degraded_area,0.152885,ha,Commodities,2020
1,Angola,AGO,2029,degraded_area,689.862428,ha,Non-Productive Conversion,2020
2,Anguilla,AIA,1915,degraded_area,0.000000,ha,Non-Productive Conversion,2020
3,Antigua and Barbuda,ATG,3095,degraded_area,0.000000,ha,Non-Productive Conversion,2020
4,Aruba,ABW,2707,degraded_area,2.701563,ha,Non-Productive Conversion,2020
...,...,...,...,...,...,...,...,...
483,Vietnam,VNM,1681,restorable_area,15828.649600,ha,Commodities,2020
484,British Virgin Islands,VGB,3037,restorable_area,24.404751,ha,Non-Productive Conversion,2020
485,"Virgin Islands, U.S.",VIR,2339,restorable_area,54.611497,ha,Non-Productive Conversion,2020
486,Wallis and Futuna,WLF,4170,restorable_area,NaN,ha,undefined,2020


In [51]:
#Degraded are values calculation might liead to NA values that translate poorly to Int during final format
degradation_driver[degradation_driver.value.isna()]

,Name,iso,id,indicator,value,unit,main_loss_driver,year
12,Bermuda,BMU,3259,degraded_area,NaN,ha,undefined,2020
24,Cook Islands,COK,3081,degraded_area,NaN,ha,undefined,2020
28,Curaçao,CUW,3075,degraded_area,NaN,ha,undefined,2020
40,French Southern Territories,ATF,3094,degraded_area,NaN,ha,undefined,2020
59,Kiribati,KIR,1867,degraded_area,NaN,ha,undefined,2020
63,Maldives,MDV,3090,degraded_area,NaN,ha,undefined,2020
64,Marshall Islands,MHL,1884,degraded_area,NaN,ha,undefined,2020
91,Sao Tome and Principe,STP,3035,degraded_area,NaN,ha,undefined,2020
97,Sint Maarten,SXM,4625,degraded_area,NaN,ha,undefined,2020
108,Togo,TGO,2855,degraded_area,NaN,ha,undefined,2020


In [52]:
print(len(degradation_driver))
print(len(degradation_driver[degradation_driver.value.notnull()]))

488
464


Table cleaning


In [53]:
degradation_final = degradation_driver[degradation_driver.value.notnull()][['id', 'indicator','year', 'value', 'unit', 'main_loss_driver']].copy()
degradation_final.value = np.int64(degradation_final.value)
degradation_final.rename(columns={'id':'location_id'}, inplace=True)
degradation_final.unit= 'ha'
degradation_final.head(3)

,location_id,indicator,year,value,unit,main_loss_driver
0,2346,degraded_area,2020,0,ha,Commodities
1,2029,degraded_area,2020,689,ha,Non-Productive Conversion
2,1915,degraded_area,2020,0,ha,Non-Productive Conversion


Save table

In [54]:
degradation_final[~degradation_final.location_id.isna()].to_csv('../../../../data/Degradation_202212.csv', index = False)

In [55]:
degradation_driver[degradation_driver.Name == 'Colombia']

,Name,iso,id,indicator,value,unit,main_loss_driver,year
20,Colombia,COL,2498,degraded_area,7072.00170,ha,Non-Productive Conversion,2020
142,Colombia,COL,2498,lost_area,17279.48978,ha,Non-Productive Conversion,2020
264,Colombia,COL,2498,mangrove_area,281190.65580,ha,Non-Productive Conversion,2020
386,Colombia,COL,2498,restorable_area,10207.48808,ha,Non-Productive Conversion,2020


In [56]:
degradation_driver.groupby('indicator').sum('value')

,id,value,year
indicator,,,
degraded_area,382080,3.561156e+05,246440
lost_area,382080,1.174439e+06,246440
mangrove_area,382080,1.472742e+07,246440
restorable_area,382080,8.183090e+05,246440
